# Aplicaciones de Aprendizaje Automático sobre Spotify 

## Propósito del documento

El objetivo del documento es explicar la aplicación de diferentes técnicas de aprendizaje automático sobre el dataset de [Spotify Tracks DB](https://www.kaggle.com/zaheenhamidani/ultimate-spotify-tracks-db). El objetivo del proyecto será estudiar diferentes configuraciones de las técnicas  para determinar la popularidad que tendrá una canción.

## Descripción del Dataset

El dataset tiene un tamaño de 32,15 MB, contiene 232,725 pistas de música. Los datos para cada una de las pistas de música son los siguientes:
- **Genre**: Género (Película, Jazz, Reggaeton...)
- **artist_name**: Nombre del artista
- **track_name**: Nombre de la pista
- **track_id**: El id de Spotify para la pista.
- **Popularity**: Ínidice de popularidad de una pista (0- 100)
- **Acousticness**: Una medida de confianza de 0.0 a 1.0 de si la pista es acústica. 1,0 representa una alta confianza de que la pista es acústica.
- **Danceability**: describe lo adecuado que es un tema para el baile basado en una combinación de elementos musicales como el tempo, la estabilidad del ritmo, la fuerza del compás y la regularidad general. Un valor de 0.0 es el menos bailable y 1.0 es el más bailable. 
- **Duration_ms**: La duración de la pista en milisegundos.  
- **Energy**: La energía es una medida de 0,0 a 1,0 y representa una medida perceptiva de la intensidad y la actividad. Típicamente, las huellas de energía se sienten rápidas, fuertes y ruidosas.
- **Instrumentalness**: Predice si una pista no contiene voces. Los sonidos "Ooh" y "aah" son tratados como instrumentales en este contexto. Las pistas de rap o de palabra hablada son claramente "vocales". 
- **Key**: La clave general estimada de la pista. Los números enteros se asignan a los lanzamientos usando la notación estándar de la clase de lanzamiento. Por ejemplo, 0 = C, 1 = C♯/D♭, 2 = D, y así sucesivamente. Si no se detectó ninguna clave, el valor es -1.
- **Liveness**: Detecta la presencia de una audiencia en la grabación ( 1 música en concierto)
- **Loudness**: La sonoridad global de una pista en decibelios (dB)
- **Mode**: El modo indica la modalidad (mayor o menor) de una pista, el tipo de escala de la que se deriva su contenido melódico. La mayor se representa por 1 y la menor por 0. 
- **Speechiness**:  detecta la presencia de palabras habladas en una pista
- **Tempo**: El tempo global estimado de una pista en pulsaciones por minuto (BPM). En la terminología musical, el tempo es la velocidad o el ritmo de una pieza dada y se deriva directamente de la duración media del tiempo
- **Time_signature**: Una firma de tiempo global estimada de una pista. La signatura de tiempo (metro) es una convención notacional para especificar cuántos latidos hay en cada barra (o medida).
- **Valence**: Una medida de 0.0 a 1.0 que describe la positividad musical transmitida por una pista (1 muy positivo)

## Librerías

In [8]:
import pandas as pd
import numpy as np

from pandas.io.parsers import read_csv
from matplotlib import pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from mpl_toolkits.mplot3d import Axes3D

## Lectura de Datos

In [4]:
data = pd.read_csv("SpotifyFeatures.csv")

In [5]:
data.shape

(232725, 18)

In [6]:
data.head(10)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.61100,0.389,99373,0.9100,0.00000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.24600,0.590,137373,0.7370,0.00000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.95200,0.663,170267,0.1310,0.00000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.70300,0.240,152427,0.3260,0.00000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.95000,0.331,82625,0.2250,0.12300,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390
5,Movie,Henri Salvador,Le petit souper aux chandelles,0Mf1jKa8eNAf1a4PwTbizj,0,0.74900,0.578,160627,0.0948,0.00000,C#,0.1070,-14.970,Major,0.1430,87.479,4/4,0.358
6,Movie,Martin & les fées,"Premières recherches (par Paul Ventimila, Lori...",0NUiKYRd6jt1LKMYGkUdnZ,2,0.34400,0.703,212293,0.2700,0.00000,C#,0.1050,-12.675,Major,0.9530,82.873,4/4,0.533
7,Movie,Laura Mayne,Let Me Let Go,0PbIF9YVD505GutwotpB5C,15,0.93900,0.416,240067,0.2690,0.00000,F#,0.1130,-8.949,Major,0.0286,96.827,4/4,0.274
8,Movie,Chorus,Helka,0ST6uPfvaPpJLtQwhE6KfC,0,0.00104,0.734,226200,0.4810,0.00086,C,0.0765,-7.725,Major,0.0460,125.080,4/4,0.765
9,Movie,Le Club des Juniors,Les bisous des bisounours,0VSqZ3KStsjcfERGdcWpFO,10,0.31900,0.598,152694,0.7050,0.00125,G,0.3490,-7.790,Major,0.0281,137.496,4/4,0.718


### Selección de variables dependientes ( x<sub>1 </sub>... x<sub>n </sub>) y Variable objetivo (y)

Las varaibles dependientes serán:
- acousticness
- danceability
- duration_ms
- energy
- instrumentalness
- key
- liveness 
- mode
- speechiness
- tempo
- time_signature
- valence
- genre

La variable objetivo será la popularidad de la canción

### Selección conjunto entrenamiento, validación, test

A lo largo del proyecto se utilizarán tres conjuntos de datos para la realización de pruebas:
- **Entrenamiento**: 70% de las pistas de música
- **Validación**: 20% de las pistas de música
- **Test**: 10% de las pistas de música

## Preprocesamiento

Antes de la elaboración de los modelos y las prueba de los mismos con diferentes configuraciones, debemos realizar un preoprocesado de los datos, en aquellas columnas que han sido seleccionadas para pertencer a los ejemplos de entrenamiento y no son númericas

#### Columna mode

Primero comprobaremos los diferentesvalores que tiene la columna

In [20]:
data['mode'].unique()

array(['Major', 'Minor'], dtype=object)

Como podemos comprobar la columna solo tiene valores Minor & Major, por ello, se colocará un 0 cuando el valor de la celda sea Minor y un 1 cuando el valor de la celda sea Major.

In [35]:
data.loc[data['mode'] == 'Minor', 'mode'] = 0
data.loc[data['mode'] == 'Major', 'mode'] = 1
data.sample(5)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
58994,Opera,Tomaso Albinoni,Albinoni : Violin Concerto in B flat major Op....,3PN4lzrD06Vqqu5S4BYLOy,27,0.9240,0.0663,171973,0.1980,0.795000,G,0.3010,-17.452,0,0.0463,62.715,4/4,0.0366
160935,Reggaeton,Dalmata,La Confrontaction,4gz7KGh5f020h2VJEvMnbG,41,0.0763,0.8430,159187,0.7460,0.000033,A,0.2620,-8.608,1,0.1120,97.998,4/4,0.9140
1966,Country,Eddie Rabbitt,Every Which Way but Loose - 2009 Remaster,73xcnqBwjnGwAlrfBomLP3,44,0.4580,0.5980,171653,0.4830,0.000008,G,0.3600,-7.474,1,0.0292,130.663,4/4,0.4520
101794,Children’s Music,Blur,Tender,42cxPm9jgbaxIVN77XA1m6,62,0.4700,0.5870,461947,0.5660,0.000000,A,0.0644,-9.932,1,0.0276,76.054,4/4,0.5290
55343,Movie,Les Hérissons,Do You Know What It Means?,0tPDastoFeurtshK6tZMYn,0,0.9730,0.6840,326227,0.0673,0.138000,G,0.1740,-17.949,1,0.0675,71.487,4/4,0.3100


#### Columna key

In [30]:
data['key'].unique()

array(['C#', 'F#', 'C', 'F', 'G', 'E', 'D#', 'G#', 'D', 'A#', 'A', 'B'],
      dtype=object)

In [32]:
key_dict = data['key'].unique()
for i in range(len(key_dict)):
    data.loc[data['key'] == key_dict[i], 'key'] = i
data.sample(5)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
3159,Alternative,grandson,War,1IqYyJ37a3gUr0NC2vOD2b,55,0.000105,0.369,211500,0.835,0.052300,0,0.2530,-4.462,0,0.0777,160.156,4/4,0.525
169097,Comedy,George Carlin,Telephone Mimes,2HManSTVHKdeuix2rauSaC,29,0.845000,0.500,69627,0.938,0.000000,10,0.8880,-13.204,1,0.9120,93.244,4/4,0.139
222004,Rock,MGMT,When You Die,3td69vL9Py7Ai9wfXYnvji,67,0.094600,0.645,263880,0.938,0.022600,11,0.1590,-4.557,0,0.0391,140.886,4/4,0.495
138992,Reggaeton,CNCO,Llegaste Tú,6tT4Ks1N04Ut0lSQ3xjSaZ,81,0.153000,0.702,190916,0.914,0.000000,4,0.0625,-2.934,0,0.0376,96.991,4/4,0.563
163019,Reggaeton,Carlitos Rossy,Ninguna,0fjousxGZT1l8Kf5HnKXkc,31,0.144000,0.689,217013,0.864,0.000027,1,0.1520,-4.736,0,0.0769,175.957,4/4,0.814


#### Columna Genre

Primero comprobaremos los diferentesvalores que tiene la columna

In [28]:
data['genre'].unique()

array(['Movie', 'R&B', 'A Capella', 'Alternative', 'Country', 'Dance',
       'Electronic', 'Anime', 'Folk', 'Blues', 'Opera', 'Hip-Hop',
       "Children's Music", 'Children’s Music', 'Rap', 'Indie',
       'Classical', 'Pop', 'Reggae', 'Reggaeton', 'Jazz', 'Rock', 'Ska',
       'Comedy', 'Soul', 'Soundtrack', 'World'], dtype=object)

Tras leer varios artículos, hemos aprendido que no es bueno que en columna con muchos posibles valores categóricos, transformarla en una columna solo con valores que representen al valor categórico, ya que los algunos modelos asignan mayor importancia a valores más altos, poor ello debemos creearnos columnas de tipo bool con nombres de los valores categóricos y poner un 1 si se trata de este valor categórico y cero en caso contrario. Procedemos a crearlas: **NO ESTOY SEGURO DE ESTO, DE MOMENTO CREO UNA SOLA COLUMNA**

In [33]:
genre_dict = data['genre'].unique()
for i in range(len(genre_dict)):
    data.loc[data['genre'] == genre_dict[i], 'genre'] = i
data.sample(5)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
111248,17,Poppy Ackroyd,Strata,0NkxSU0b1Jsw6yK2mJZru3,64,0.9910,0.320,273165,0.0773,0.909000,5,0.0750,-27.053,0,0.0644,73.589,4/4,0.0368
89396,11,YFN Lucci,Boss Life (feat. Offset),1yWas3061dbaZF9Zq6kfKZ,52,0.0757,0.775,188631,0.4260,0.000000,0,0.1240,-6.583,1,0.1710,145.154,4/4,0.3220
117923,14,Ca$h Out,Cashin' Out,1POAx4NMLOBPVKZUSsBh92,57,0.1490,0.831,238760,0.6650,0.000000,0,0.0636,-3.627,0,0.0711,129.989,4/4,0.6630
197424,24,Slum Sociable,Treated Like The Weather,7FYwxdrJX2k5JtFUJMGItR,46,0.3720,0.574,250796,0.5600,0.013200,3,0.2060,-6.338,1,0.0344,97.999,4/4,0.3550
116244,14,Mike Posner,Song About You,29dfnKJ6WvYcJpgQ0MKzX7,62,0.1800,0.676,190147,0.8490,0.000024,8,0.0998,-3.008,1,0.0412,87.025,4/4,0.6410


## Regresión Lineal

- Separamos datos en entrenamiento, validación y test
- Normalizamos datos
- Averiguamos coste y thethas optimas (linearCostGrad y después optimize)
- ¿Se usa la funcion optimice de scipy (como en la practica 5 de Regresión Lineal Regularizada)? ¿O se hace todo con nuestro código como en la práctica 1?
- Pintamos error en datos de entrenamiento y de validación.
- Elegimos parámetro de regularización.
- Creamos modelo.


In [ ]:
def cost(thetas, X, Y, reg=0):
    m = X.shape[0]
    H = np.dot(X, thetas)
    cost = (1/(2*m)) * np.sum((H-Y.T)**2) + ( reg / (2 * m) ) * np.sum(thetas[1:]**2)
    return cost

def gradient(thetas, X, Y, reg=0):
    tt = np.copy(thetas)
    tt[0]=0
    m = X.shape[0]
    H = np.dot(X, thetas)
    gradient = ((1 / m) * np.dot(H-Y.T,X)) + ((reg/m) * tt)
    return gradient

def linearCostGrad(thetas,X,Y,reg=0):
    return (cost(thetas,X,Y,reg),gradient(thetas,X,Y).flatten())

def polinomial_data(X,p):
    X_poly = X
    for i in range(1, p):
        X_poly = np.column_stack((X_poly, np.power(X, i+1)))   
    return X_poly

def normalize(X):
    medias = np.mean(X,axis=0)
    sigmas = np.std(X,axis=0)
    
    X_normalizada = (X - medias) / sigmas
    
    return (X_normalizada, medias, sigmas)

## Regresión Logística

## Redes Neuronales

## Máquinas de vector de soporte

# DUDAS

- En columna género ¿Crear solo una columna de float o 27 columnas de bool?
- Metemos el nombvre del artista, hay 14500 diferentes ?¿